# Install libraries

In [ ]:
import requests
import pandas as pd
import plotly.express as px
import folium
import plotly.io as pio
import time
import random
from tqdm import tqdm

# API CEREMA

In [ ]:
BASE_URL_API = "https://apidf-preprod.cerema.fr"

In [ ]:
def apidf(url_endpoint, token=None):
    HEADERS = {
        "Content-Type": "application/json",
    }
    if token:
        HEADERS["Authorization"] = "Token " + token
    response = requests.get(
        url_endpoint,
        headers=HEADERS,
    )
    if response.status_code == 200:
      return response.json()
    return None

In [ ]:
# List of fake User-Agents
user_agents =[
    "Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/532.1 (KHTML, like Gecko) Chrome/4.0.213.0 Safari/532.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.1 (KHTML, like Gecko) Chrome/4.0.213.0 Safari/532.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.1 (KHTML, like Gecko) Chrome/4.0.213.0 Safari/532.1",
    "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_0; en-US) AppleWebKit/532.1 (KHTML, like Gecko) Chrome/4.0.212.1 Safari/532.1",
    "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_5_7; en-US) AppleWebKit/532.1 (KHTML, like Gecko) Chrome/4.0.212.1 Safari/532.1",
    "Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.212.0 Safari/532.0",
    "Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/532.1 (KHTML, like Gecko) Chrome/4.0.212.0 Safari/532.1",
    "Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.212.0 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.212.0 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.212.0 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.212.0 Safari/532.0",
    "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.212.0 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.7 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.7 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.4 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.4 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.4 Safari/532.0",
    "Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.2 Safari/532.0",
    "Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.2 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.2 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.2 Safari/532.0",
    "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.2 Safari/532.0",
    "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_5_8; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.2 Safari/532.0",
    "Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.0 Safari/532.0",
    "Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.0 Safari/532.0",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.0 Safari/532.0"]

# Function to get a random user agent
def get_random_user_agent():
    return random.choice(user_agents)

In [ ]:
# Codes Insee Paris
codinsee = ['75101','75102','75103','75104','75105','75106','75107','75108','75109','75110','75111','75112','75113','75114','75115','75116','75117','75118','75119','75120']
donnees_mutations = []
visited_mutations = set()
x, y = 2.340976, 48.862033
m = folium.Map(location=[y, x], zoom_start=16)

with tqdm(total=len(codinsee), desc='Overall Progress') as overall_progress:
    for code in codinsee:

        overall_progress.update(1)

        url = BASE_URL_API + f"/dvf_opendata/geomutations/?code_insee={code}&anneemut_min=2020&codtypbien=12"
        #retries = {}

        while True:
            try:
                user_agent = get_random_user_agent()
                headers = {'User-Agent': user_agent}
                response = requests.get(url,
                                        headers=headers,
                                        timeout=(5,30)).json()

                folium.GeoJson(response,
                               name="mutations",
                               popup=folium.GeoJsonPopup(fields=["datemut", "valeurfonc","sbati"])).add_to(m)

                for feature in response['features']:
                    properties = feature['properties']
                    mutation_id = properties["idmutinvar"]
                    mutation_info = {
                        "idmutinvar": properties["idmutinvar"],
                        "libtypbien": properties["libtypbien"],
                        "anneemut": properties["anneemut"],
                        "datemut": properties["datemut"],
                        "coddep": properties["coddep"],
                        "l_codinsee": properties.get("l_codinsee"),
                        "valeurfonc": properties["valeurfonc"],
                        "l_idpar": properties.get("l_idpar"), # parcelle cadastrale associée
                        "sbati": properties.get("sbati"),  # surface batiment au m2
                    }

                    if mutation_id not in visited_mutations:
                        donnees_mutations.append(mutation_info)
                        visited_mutations.add(mutation_id)

                if not response["next"]:
                    break
                url = response["next"]

            except Exception as e:
                print(f"Erreur pour le code INSEE {code}: {mutation_id}")

                time.sleep(random.uniform(1, 10))
                continue

Overall Progress:  20%|██        | 4/20 [00:59<04:43, 17.69s/it]

Erreur pour le code INSEE 75104: 0aa837a3484d055a282d832697c09c53


Overall Progress:  30%|███       | 6/20 [02:09<06:24, 27.46s/it]

Erreur pour le code INSEE 75106: d2f57b47848e9d2a20698a652437616b
Erreur pour le code INSEE 75106: 6b07a1fa06d8709feb47c66ec2ae4639


Overall Progress:  55%|█████▌    | 11/20 [06:17<07:30, 50.03s/it]

...

Overall Progress:  95%|█████████▌| 19/20 [26:18<02:46, 166.81s/it]

Erreur pour le code INSEE 75119: 3324a6e716c0871604952aceb7d91401
Erreur pour le code INSEE 75119: 1011fd229429c20b1405858473639717
Erreur pour le code INSEE 75119: f962d72dcee8c5670a86a1c3788fe3c9
Erreur pour le code INSEE 75119: 42104162e00c2fbea453119c5c04a661


Overall Progress: 100%|██████████| 20/20 [28:18<00:00, 152.47s/it]

Erreur pour le code INSEE 75120: 4af1f05b9791e69cf768e72d4d6ac732
Erreur pour le code INSEE 75120: 9da57840319a8d7da0f810442d9f2dae


Overall Progress: 100%|██████████| 20/20 [30:07<00:00, 90.36s/it] 


In [ ]:
df_mutations = pd.DataFrame()
df_mutations= pd.DataFrame(donnees_mutations)

# Verification and data preprocessing

In [ ]:
# delete the lists from the dataframe
df_mutations['l_codinsee'] = df_mutations['l_codinsee'].apply(lambda x: x[0] if isinstance(x, list) else x)
df_mutations['l_idpar'] = df_mutations['l_idpar'].apply(lambda x: x[0] if isinstance(x, list) else x)

In [ ]:
# Are there any duplicates ? No
doublons = df_mutations[df_mutations.duplicated()]
print("Lignes en double :\n", doublons)

Lignes en double :
 Empty DataFrame
Columns: [idmutinvar, libtypbien, anneemut, datemut, coddep, l_codinsee, valeurfonc, l_idpar, sbati]
Index: []


In [ ]:
# Are there missing values ? Yes some valeurfonc missing --> delete
df_mutations.isna().sum()

idmutinvar     0
libtypbien     0
anneemut       0
datemut        0
coddep         0
l_codinsee     0
valeurfonc    46
l_idpar        0
sbati          0
dtype: int64

# Add coordinates of each parcelles

In [ ]:
import gzip
import json

chemin_general = "drive/MyDrive/Colab Notebooks/Data science project/"

with gzip.open(chemin_general + "BDD_input/cadastre-75-parcelles.json.gz", 'rb') as f:
    data = f.read().decode('utf-8')
    json_data = json.loads(data)

In [ ]:
from shapely.geometry import Polygon

id_parcelles = []
mean_coor_parcelles = []

for feature in json_data['features']:
    id_parcelles.append(feature['id'])
    coordinates = feature['geometry']['coordinates'][0]

    polygon = Polygon(coordinates)
    # Calculate the center of polygon
    centroid = polygon.centroid
    mean_coordinate = [centroid.y, centroid.x]

    mean_coor_parcelles.append(mean_coordinate)

df_parcelles = pd.DataFrame({'id_parcelles': id_parcelles, 'coor_parcelles': mean_coor_parcelles})

In [ ]:
mutation_final = pd.merge(df_mutations, df_parcelles, how='left', left_on='l_idpar', right_on='id_parcelles').drop('l_idpar', axis=1)

In [ ]:
mutation_final.isna().sum()

idmutinvar        0
libtypbien        0
anneemut          0
datemut           0
coddep            0
l_codinsee        0
valeurfonc        0
sbati             0
id_parcelles      0
coor_parcelles    0
dtype: int64

In [ ]:
mutation_final = mutation_final.dropna()

In [ ]:
mutation_final['long_parcelles'] = mutation_final['coor_parcelles'].map(lambda x: x[0])
mutation_final['lat_parcelles'] = mutation_final['coor_parcelles'].map(lambda x: x[1])
mutation_final = mutation_final.reset_index(drop=True)

# Save data

In [ ]:
# save the map in html
m.save('mutations.html')

KeyboardInterrupt: 

In [ ]:
# save data in CSV
chemin_general = "drive/MyDrive/Colab Notebooks/Data science project/"
mutation_final.to_csv(chemin_general + "BDD_output/mutations.csv", index=False)